# Recover the Homography with DLT in the Absence of Noise

A 2D canvas shows two houses. One house is generated by applying a projetivity on the other house.
Please determine the homography matrix.
  
Hint: Please make sure that you have installed Scipy. You can do that by entering
`python.exe -m pip install --user scipy`
on the CMD or Powershell.

In [1]:
# Do not learn imports by hard for the exam in general. This is not important.
import numpy as np
import scipy as sp
import scipy.linalg

# use the following 2-by-11 matrix to define a simple house
X_inhom = np.array([
    [-6, -6, -7, 0, 7, 6,  6, -3, -3,  0,  0],
    [-7,  2,  1, 8, 1, 2, -7, -7, -2, -2, -7]
], dtype=np.float)

# make points homogeneous
num_pts = X_inhom.shape[1]
X = np.concatenate([X_inhom, np.ones((1, num_pts))])

# determine the exact solution -> only 4 point correnspondences are required
# choose 4 points of X
subset = [0, 2, 4, 6]
num_pts_sel = len(subset)
assert(num_pts_sel == 4)

Now, let's define and apply a homography that should be recovered later on.
Euclidean transformations are a subset of affinities and affinities are a subset of projectivities.
Let's use a simple rotation as our homography.

In [2]:
# 2D rotation homogeneous space
alpha = 45 * np.pi / 180 # 45 degrees

H = np.array([
    [np.cos(alpha), -np.sin(alpha), 0],
    [np.sin(alpha),  np.cos(alpha), 0],
    [0,           0,          1]
], dtype=np.float)

X2 = H.dot(X)

As mentioned, we only need 4 point correspondences if there is no noise. We have already made a selection of 4 points by defining the python list `subset`.

In [7]:
X_sel  =  X[:, subset]
X2_sel = X2[:, subset]

print (X2_sel)

[[ 0.70710678 -5.65685425  4.24264069  9.19238816]
 [-9.19238816 -4.24264069  5.65685425 -0.70710678]
 [ 1.          1.          1.          1.        ]]


Remember, the DLT linear equation system of the lecture:  
$\begin{pmatrix}
    \mathbf{0}^T & -w'_i\mathbf{x}^T_i & y'_i\mathbf{x}^T_i \\
    w'_i\mathbf{x}^T_i & \mathbf{0}^T & -x'_i\mathbf{x}^T_i \\
\end{pmatrix} \cdot
\begin{pmatrix}
    \mathbf{h}_{\text{r}1} \\ \mathbf{h}_{\text{r}2} \\ \mathbf{h}_{\text{r}3}
\end{pmatrix}
= \mathbf{0}$  
  
$ \mathbf{A}_i \cdot \mathbf{h} = \mathbf{0} $

  
Whereas $\mathbf{x}_i$ is the $i$-th point in the point array `X_sel` and 
$\mathbf{x}'_i = \begin{pmatrix} x'_i \\ y'_i \\ w'_i \end{pmatrix}$ is the projectively transformed $i$-th point in the point array `X2_sel`.  
$\mathbf{h}_{\text{r}i}$ is `H[i,:].T`, the transposed $i$-th row of the homography $\mathbf{H}$.
  
For 4 point correspondences, $\mathbf{A}$ is the concatenation of all $\mathbf{A}_i \text{ for } i \in \{1,2,3,4\}$.
You might want to reorganize the order of colums (wihtout loss of generality):  
$\begin{pmatrix}
    \mathbf{0}^T & -w'_1\mathbf{x}^T_1 & y'_1\mathbf{x}^T_1 \\
    \mathbf{0}^T & -w'_2\mathbf{x}^T_2 & y'_2\mathbf{x}^T_2 \\
    \mathbf{0}^T & -w'_3\mathbf{x}^T_3 & y'_3\mathbf{x}^T_3 \\
    \mathbf{0}^T & -w'_4\mathbf{x}^T_4 & y'_4\mathbf{x}^T_4 \\
    w'_1\mathbf{x}^T_1 & \mathbf{0}^T & -x'_1\mathbf{x}^T_1 \\
    w'_2\mathbf{x}^T_2 & \mathbf{0}^T & -x'_2\mathbf{x}^T_2 \\
    w'_3\mathbf{x}^T_3 & \mathbf{0}^T & -x'_3\mathbf{x}^T_3 \\
    w'_4\mathbf{x}^T_4 & \mathbf{0}^T & -x'_4\mathbf{x}^T_4 \\
\end{pmatrix} \cdot
\begin{pmatrix}
    \mathbf{h}_{\text{r}1} \\ \mathbf{h}_{\text{r}2} \\ \mathbf{h}_{\text{r}3}
\end{pmatrix}
= \mathbf{0}$  
  
$ \begin{pmatrix}
    \mathbf{A}_\text{top} \\ \mathbf{A}_\text{bottom}
\end{pmatrix}
\cdot \mathbf{h} = \mathbf{0} $  
  
$ \mathbf{A} \cdot \mathbf{h} = \mathbf{0} $
  
Now, consider the first 4 rows of $\mathbf{A}$ being `A_top` and the last 4 rows of $\mathbf{A}$ being `A_bottom`.


In [8]:
x_trans = X2_sel[0,:]
y_trans = X2_sel[1,:]
w_trans = X2_sel[2,:]
print("shape of x' is: ", x_trans.shape)
print(x_trans)

shape of x' is:  (4,)
[ 0.70710678 -5.65685425  4.24264069  9.19238816]


In [9]:
# We need (4,1) not (4,).
# Sometimes numpy squeezes dimensions. Let's undo this:
x_trans = np.expand_dims(x_trans, axis=1)
y_trans = np.expand_dims(y_trans, axis=1)
w_trans = np.expand_dims(w_trans, axis=1)
print("shape of x' is: ", x_trans.shape)
print(x_trans)

shape of x' is:  (4, 1)
[[ 0.70710678]
 [-5.65685425]
 [ 4.24264069]
 [ 9.19238816]]


In [10]:
row_1234_col_123 = np.zeros((num_pts_sel, 3), dtype=np.float)
row_1234_col_456 = -w_trans * X_sel.T
row_1234_col_789 =  y_trans * X_sel.T

# np.vstack(...) is equivalent to np.concatenate(..., axis=0) and means stack vertically
# np.hstack(...) is equivalent to np.concatenate(..., axis=1) and means stack horizontally

A_top = np.hstack([row_1234_col_123, row_1234_col_456, row_1234_col_789])
print("shape of A_top is:", A_top.shape)

row_5678_col_123 = w_trans * X_sel.T
row_5678_col_456 = np.zeros((num_pts_sel, 3), dtype=np.float)
row_5678_col_789 = -x_trans * X_sel.T

A_bottom = np.hstack([row_5678_col_123, row_5678_col_456, row_5678_col_789])
print("shape of A_bottom is:", A_bottom.shape)

A = np.vstack([A_top, A_bottom])
print("shape of A is:", A.shape)

shape of A_top is: (4, 9)
shape of A_bottom is: (4, 9)
shape of A is: (8, 9)


Since $\mathbf{A} \cdot \mathbf{h} = \mathbf{0}$, $\mathbf{h}$ is the right null space of $\mathbf{A}$.  
We can use Scipy's null space method to determine the solution.

In [11]:
h = sp.linalg.null_space(A)
H_r = h.reshape((3, 3)) # from vector back to matrix
H_r = H_r / H_r[-1, -1] # divide by last row and last collumn to normalize the homography. Now, H[3, 3] should be 1

np.set_printoptions(suppress=True)
print("The reconstructed homography is:")
print(H_r)

The reconstructed homography is:
[[ 0.70710678 -0.70710678 -0.        ]
 [ 0.70710678  0.70710678 -0.        ]
 [-0.          0.          1.        ]]


How big is the reconstruction error?

In [12]:
# calculate the MAE (mean absolute error)
residuum = H - H_r
print("residuum:", residuum)
abs_error = np.abs(residuum)
mae = np.mean(abs_error)
print("MAE:", mae)

residuum: [[ 0.  0.  0.]
 [-0.  0.  0.]
 [ 0.  0.  0.]]
MAE: 9.820143689476782e-16


How to interpret very small numbers like this?
For instance, `5e-16` means $5 \cdot 10^{-16}$.